# DSL development

## Imports

In [ ]:
# Use this to reload changes in python scripts
%reload_ext autoreload
%autoreload 2

In [ ]:
import random
import logging
import numpy as np
from IPython.display import Markdown, display
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.auto import tqdm

from arc25.dsl import *
from arc25.training_tasks import *
from arc25.plot import plot_task
from arc25.logging import configure_logging

configure_logging(level=logging.INFO)
logging.getLogger('matplotlib').setLevel(logging.CRITICAL)


plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (20, 3)  
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 12


## Drawing samples

I want to generate training data so the model can learn how to draw things. It is very easy to generate
synthetic training data and learning about it should provide the model an understanding of the 2d structure of the grids.

In [ ]:
task_generator = RandomDrawingTaskOnEmptyImg(min_draws=5)

for _ in range(3):
    task = task_generator.sample()
    display(Markdown("```python\n" + task.code + "\n```"))
    plot_task(task)

In [ ]:
task

In [ ]:
#[task_generator.sample() for _ in tqdm(range(4000), smoothing=0)];

Without validation I'm able to generate 2500 tasks per second. When adding code validation I generate around 1000 tasks per second.